# Multiple inheritance in Python

This is a simple example to illustrate the concepts associated to Multiple inheritance in Python.
We want to create a new class based on a the Python built-in `dict`.

In [1]:
import collections
from pprint import pprint
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

class LoggingDict(dict):
    
    def __init__(self, **kwargs):
        logging.info('Initializing my dict')
        super().__init__(**kwargs)  # Could be dict.__init__(self, **kwargs)

    def __setitem__(self, key, value):
        logging.info('Setting key=%s, value=%s' % (key, value))
        super().__setitem__(key, value)  # Could be dict.__setitem__(self, key, value)

It is quite straightforward, you've probably seen very similar uses of `super` in `__init__` functions. 

Let's instantiate it and set an item:

In [2]:
ld = LoggingDict()
ld['a'] = 42

INFO:root:Initializing my dict
INFO:root:Setting key=a, value=42


It is working as intended! But for all that we care now, we could also have used `dict.__setitem__(self, key, value)`. Admittedly the `super` syntax looks much nicer, but is it its only purpose?

We can see a first advantage of `super()`. We are refering to the superclass `dict` by the built-in `super`, which means that if the name of the base class somehow changes, the code of `LoggingDict` remains unbroken. This is called **indirection** or **dereferencing**.

## Indirection in practice

Let's say we want now to log and `OrderedDict`, the only thing we have to do is this:

In [3]:
class LoggingOrderedDict(LoggingDict, collections.OrderedDict):
    pass

In [4]:
lod = LoggingOrderedDict()
lod['b'] = 23

INFO:root:Initializing my dict
INFO:root:Setting key=b, value=23


Note that if we call `__dict__.setiem__(self,key,value)` instead of `super().__setitem(key, value)` in our `LoggingDict`, then obvisously the ordered feature of our `LoggingOrderedDict` will be lost, as it is indeed `OrderedDidct.__setitem__` that needs to be called, not `dict__setitem__`!

So the indirection is not just a matter of forward compatibility, it also just saved us the need to recreate a very similar class `LoggingOrderedDict` that would only have inherited only from `collections.OrderedDict`.

## Order matters

Why is this working? Becaues by giving the LoggingDict in first position, we ensured that any method called from a `LoggingOrderedDict` instance will be called in the following order:

In [5]:
LoggingOrderedDict.__mro__

(__main__.LoggingOrderedDict,
 __main__.LoggingDict,
 collections.OrderedDict,
 dict,
 object)

Indeed, if we define the `LoggingOrderedDict` in the other order:

In [6]:
class LoggingOrderedDict(collections.OrderedDict, LoggingDict):
    pass
LoggingOrderedDict.__mro__

(__main__.LoggingOrderedDict,
 collections.OrderedDict,
 __main__.LoggingDict,
 dict,
 object)

We can see that the `__setitem__` method will be fetched from `OrderedDict` first, which does not have our neat logging feature:

In [7]:
lod = LoggingOrderedDict()
lod['b'] = 23

Indeed, no log is printed.

## Further readings

 * https://rhettinger.wordpress.com/2011/05/26/super-considered-super/
 * https://stackoverflow.com/a/27134600/8597378'
 * On the actual algorithm: https://en.wikipedia.org/wiki/C3_linearization
 * On indirection or dereferencing: https://en.wikipedia.org/wiki/Indirection